# Objetivo

O objetivo dessa atividade é simular um cenário real de limpeza de dados, fazendo uma análise completa das variáveis e, posteriormente, importar para o Power BI.

In [1]:
import pandas as pd
import calendar
import openpyxl

from datetime import datetime

In [2]:
# Importando os dados
df = pd.read_csv("./vendas.csv")

df.head(10)

,id_transacao,data_venda,produto_categoria,valor_unitario,quantidade,metodo_pagamento,valor_total
0,1001,2025-12-25,Eletrônicos,3846.58,1,Pix,3846.58
1,1002,2025-03-04,NaN,216.95,3,pix,650.85
2,1003,2025-11-02,Eletrodomésticos,4911.77,5,Cartão de Crédito,24558.85
3,1004,2025-12-06,Eletrodomésticos,854.3,2,Transferência,1708.60
4,1005,2025-07-01,Eletrônicos,1718.19,2,Boleto,3436.38
5,1006,2025-10-24,ELETRÔNICOS,3597.4,2,pix,7194.80
6,1007,2025-10-15,Informática,3224.24,3,Pix,9672.72
7,1008,2025-04-14,Eletrônicos,4248.47,5,Transferência,21242.35
8,1009,2025-06-11,Informática,2781.06,3,Transferência,8343.18
9,1010,2025-12-22,Eletrônicos,354.33,4,Transferência,1417.32


In [3]:
# Há valores nulos para serem tratados.

df.isnull().sum()

id_transacao          0
data_venda            0
produto_categoria    14
valor_unitario       15
quantidade            0
metodo_pagamento      0
valor_total           0
dtype: int64

In [4]:
# Existem variáveis para alterar o dtype.

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305 entries, 0 to 304
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id_transacao       305 non-null    int64  
 1   data_venda         305 non-null    object 
 2   produto_categoria  291 non-null    object 
 3   valor_unitario     290 non-null    object 
 4   quantidade         305 non-null    int64  
 5   metodo_pagamento   305 non-null    object 
 6   valor_total        305 non-null    float64
dtypes: float64(1), int64(2), object(4)
memory usage: 16.8+ KB


# Tratamento dos dados

## Valores NaN

>O que fazer com os valores ```NaN```?

Para responder essa pergunta, temos que analisar os seguintes aspectos:
- plano de negócio,
- dimensão dos dados,
- impacto da exclusão (ou não) desses valores no banco de dados,
- como substituir, se preciso, esses valores? Média? Mediana? Log?

### Entendendo os valores NaN
Os valores se encontram nas colunas **produto_categoria** (*qualitativa*) e **valor_unitario** (*quantitativa*).


---

## produto_categoria
Existem algumas categorias em caixa alta, essas também deverão ser tratadas.

---

## Dtypes
As colunas ```data_venda``` e ```valor_unitario``` estão com o dtype incorretos. O primeiro, deverá ser transformado para *datetime*, e o segundo para *float*.

---

# NaN

In [5]:
""" 
Sintaxe para apresentar os valores NaN da coluna produto_categoria. Pensando no plano de negócio, não podemos excluir essas colunas.
Visto que a exclusão poderá retirar informações importantes como método de pagamento mais utilizado, valor total das compras etc.
Para manter essas informações, irei adicionar uma nova categoria para os valores NaN, chamada 'indefinida'.
"""

df[df['produto_categoria'].isnull()]

,id_transacao,data_venda,produto_categoria,valor_unitario,quantidade,metodo_pagamento,valor_total
1,1002,2025-03-04,NaN,216.95,3,pix,650.85
15,1016,2025-01-29,NaN,1015.17,5,Boleto,5075.85
54,1055,2025-07-19,NaN,4430.02,4,Cartão de Crédito,17720.08
104,1105,2025-12-26,NaN,2349.64,3,Transferência,7048.92
144,1145,2025-09-02,NaN,1783.65,1,pix,1783.65
145,1146,2025-08-26,NaN,211.14,1,Cartão de Crédito,211.14
156,1157,2025-01-23,NaN,3737.4,1,Pix,3737.40
159,1160,2025-05-31,NaN,2655.74,1,Cartão de Crédito,2655.74
182,1183,2025-09-14,NaN,2542.81,3,Boleto,7628.43
209,1210,2025-07-18,NaN,2778.76,2,Boleto,5557.52


In [6]:
"""
A função fillna preenche os valores NaN com o parâmetro entre parênteses.
"""

df['produto_categoria'] = df['produto_categoria'].fillna('Indefinida')
df

,id_transacao,data_venda,produto_categoria,valor_unitario,quantidade,metodo_pagamento,valor_total
0,1001,2025-12-25,Eletrônicos,3846.58,1,Pix,3846.58
1,1002,2025-03-04,Indefinida,216.95,3,pix,650.85
2,1003,2025-11-02,Eletrodomésticos,4911.77,5,Cartão de Crédito,24558.85
3,1004,2025-12-06,Eletrodomésticos,854.3,2,Transferência,1708.60
4,1005,2025-07-01,Eletrônicos,1718.19,2,Boleto,3436.38
...,...,...,...,...,...,...,...
300,1001,2025-12-25,Eletrônicos,3846.58,1,Pix,3846.58
301,1002,2025-03-04,Indefinida,216.95,3,pix,650.85
302,1003,2025-11-02,Eletrodomésticos,4911.77,5,Cartão de Crédito,24558.85
303,1004,2025-12-06,Eletrodomésticos,854.3,2,Transferência,1708.60


In [7]:
"""
Valores NaN em colunas monetárias merecem uma análise mais detalhada, pois existem N maneiras de substituir esses valores.
A média ou a mediana fariam sentido nesse contexto? Talvez não tragam as informações mais fidedignas.
Neste caso, temos as colunas 'quantidade' e 'valor_total' que, realizando a divisão, trazem o valor unitário real.
"""

df[df['valor_unitario'].isnull()]

,id_transacao,data_venda,produto_categoria,valor_unitario,quantidade,metodo_pagamento,valor_total
28,1029,2025-09-04,ELETRÔNICOS,NaN,2,Cartão de Crédito,2608.44
68,1069,2025-08-09,Acessórios,NaN,2,Boleto,1133.08
73,1074,2025-08-20,Eletrônicos,NaN,5,Transferência,9422.90
90,1091,2025-05-18,Smartphones,NaN,4,pix,12593.76
115,1116,2025-12-10,ELETRÔNICOS,NaN,4,Cartão de Crédito,13215.32
116,1117,2025-10-23,Informática,NaN,1,Transferência,3438.04
121,1122,2025-02-15,ELETRÔNICOS,NaN,3,Boleto,713.40
153,1154,2025-10-27,Acessórios,NaN,2,Cartão de Crédito,4697.30
176,1177,2025-11-02,Eletrônicos,NaN,4,Cartão de Crédito,8402.96
216,1217,2025-09-03,ELETRÔNICOS,NaN,3,Transferência,7934.79


In [8]:
"""
Substituindo os valores NaN pela divisão entre total / quantidade.
"""

df['valor_unitario'] = df['valor_unitario'].fillna(df['valor_total'] / df['quantidade'])

# Não há valores NaN.
df[df['valor_unitario'].isnull()]

,id_transacao,data_venda,produto_categoria,valor_unitario,quantidade,metodo_pagamento,valor_total


# produto_categoria (em caixa alta)

In [9]:
df['produto_categoria'] = df['produto_categoria'].str.title()
df.head(10)

,id_transacao,data_venda,produto_categoria,valor_unitario,quantidade,metodo_pagamento,valor_total
0,1001,2025-12-25,Eletrônicos,3846.58,1,Pix,3846.58
1,1002,2025-03-04,Indefinida,216.95,3,pix,650.85
2,1003,2025-11-02,Eletrodomésticos,4911.77,5,Cartão de Crédito,24558.85
3,1004,2025-12-06,Eletrodomésticos,854.3,2,Transferência,1708.60
4,1005,2025-07-01,Eletrônicos,1718.19,2,Boleto,3436.38
5,1006,2025-10-24,Eletrônicos,3597.4,2,pix,7194.80
6,1007,2025-10-15,Informática,3224.24,3,Pix,9672.72
7,1008,2025-04-14,Eletrônicos,4248.47,5,Transferência,21242.35
8,1009,2025-06-11,Informática,2781.06,3,Transferência,8343.18
9,1010,2025-12-22,Eletrônicos,354.33,4,Transferência,1417.32


# Alterando o dtype

Ao tentar transformar a variável, surgiu um erro indicando que há valores fora do padrão ```"%Y-%m-%d"```.

Para tratar isso, usarei o método ```errors='coerce'```, ele transformará datas inválidas em **NaT**, depois conseguirei visualizar e tratá-las.

In [10]:
df['data_venda'] = pd.to_datetime(df['data_venda'])
df.dtypes

ValueError: time data "31/02/2025" doesn't match format "%Y-%m-%d", at position 51. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

## Análise da data_venda

Todos os valores incorretos estão no formato ```dia/mês/ano```, sendo que o formato correto é ```ano-mês-dia```.

Outra questão é a data em si, visto que fevereiro não possui 31 dias.

---

### Tratando
Usarei uma função que tenta converter como ```ano-mês-dia```, se falhar, tenta dividir como ```dia/mês/ano```, se o dia for inválido, ajusta para o último dia do mês. Se ainda falhar, retorna **NaT**.


In [11]:
# Cópia de segurança para o df
df_analise = df.copy()

# Criei uma nova coluna idêntica a data_venda
df_analise['data_convertida'] = df_analise['data_venda']

# Aqui faço a transformação das datas incorretas para NaT, assim poderei comparar as duas colunas posteriormente.
df_analise['data_convertida'] = pd.to_datetime(
    df_analise['data_convertida'],
    format='%Y-%m-%d',
    errors='coerce'
)

# Apresentando os dados com NaT em data_convertida.
df_analise[df_analise['data_convertida'].isna()]

,id_transacao,data_venda,produto_categoria,valor_unitario,quantidade,metodo_pagamento,valor_total,data_convertida
51,1052,31/02/2025,Informática,227.38,2,Boleto,454.76,NaT
113,1114,31/02/2025,Acessórios,1472.57,3,Boleto,4417.71,NaT
117,1118,31/02/2025,Eletrônicos,3079.07,2,Pix,6158.14,NaT
122,1123,31/02/2025,Smartphones,561.83,4,Boleto,2247.32,NaT
124,1125,31/02/2025,Smartphones,1863.69,5,Boleto,9318.45,NaT
128,1129,31/02/2025,Eletrodomésticos,912.34,3,Cartão de Crédito,2737.02,NaT
129,1130,31/02/2025,Eletrodomésticos,2152.61,3,Transferência,6457.83,NaT
166,1167,31/02/2025,Acessórios,3058.89,5,Cartão de Crédito,15294.45,NaT
196,1197,31/02/2025,Eletrônicos,4107.77,5,Cartão de Crédito,20538.85,NaT
203,1204,31/02/2025,Informática,1659.08,3,Boleto,4977.24,NaT


In [12]:
def corrigir_data(data_str): # Cria uma função
    try:                                                    # Tenta converter a string usando o formato padrão
        return datetime.strptime(data_str, "%Y-%m-%d")      # Se funcionar, retorna a data convertida
    except:                                                 # Se der erro, entra nesse bloco
        try:                                                # Bloco da correção automática
            dia, mes, ano = map(int, data_str.split("/"))   # Divide a string pelo /, converte cada parte para inteiro e atribuí às variáveis dia, mes, ano.
            ultimo_dia = calendar.monthrange(ano, mes)[1]   # Descobri o último dia do mês
            return datetime(ano, mes, ultimo_dia)           # Cria uma nova data corrigida
        except:                                             # Se TUDO falhar
            return pd.NaT                                   # Retorna valor ausente

df['data_venda_tratada'] = df['data_venda'].apply(corrigir_data)

## valor_unitario

Ao tentar converter a variável, surgiu o erro de formatação, há valores como esse 'R$ 1.500,00'.

---

### Tratamento

Vou aplicar uma substituição usando a fórmula:
```Latex
valor_total / quantidade 
```

In [13]:
df['valor_unitario'] = df['valor_unitario'].astype(float)

ValueError: could not convert string to float: 'R$ 1.500,00'

In [14]:
df[df['valor_unitario'] == 'R$ 1.500,00']

,id_transacao,data_venda,produto_categoria,valor_unitario,quantidade,metodo_pagamento,valor_total,data_venda_tratada
36,1037,2025-02-20,Eletrônicos,"R$ 1.500,00",3,Cartão de Crédito,11672.49,2025-02-20
64,1065,2025-02-14,Eletrônicos,"R$ 1.500,00",3,Transferência,2303.97,2025-02-14
84,1085,2025-06-29,Informática,"R$ 1.500,00",4,Transferência,2433.92,2025-06-29
99,1100,2025-09-12,Informática,"R$ 1.500,00",3,pix,11612.25,2025-09-12
105,1106,2025-07-03,Eletrônicos,"R$ 1.500,00",3,Boleto,11006.19,2025-07-03
125,1126,2025-01-11,Informática,"R$ 1.500,00",3,Cartão de Crédito,3826.77,2025-01-11
149,1150,2025-07-14,Eletrodomésticos,"R$ 1.500,00",2,Pix,3442.78,2025-07-14
253,1254,2025-04-17,Smartphones,"R$ 1.500,00",5,Pix,13457.30,2025-04-17
276,1277,2025-10-16,Acessórios,"R$ 1.500,00",5,Cartão de Crédito,17834.25,2025-10-16
278,1279,31/02/2025,Eletrônicos,"R$ 1.500,00",1,Cartão de Crédito,4844.65,2025-02-28


In [15]:
df['valor_unitario'] = df['valor_total'] / df['quantidade']

In [23]:
"""
O valor foi substituído pelo correto.
"""

df.iloc[36, 3]

3890.83

In [26]:
df['valor_unitario'] = df['valor_unitario'].astype(float)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305 entries, 0 to 304
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id_transacao        305 non-null    int64         
 1   data_venda          305 non-null    object        
 2   produto_categoria   305 non-null    object        
 3   valor_unitario      305 non-null    float64       
 4   quantidade          305 non-null    int64         
 5   metodo_pagamento    305 non-null    object        
 6   valor_total         305 non-null    float64       
 7   data_venda_tratada  305 non-null    datetime64[ns]
 8   erro_calculo        305 non-null    float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(3)
memory usage: 21.6+ KB


# Valores monetários

Outro ponto que irei analisar serão os valores monetários, para confirmar se os dados estão corretos.

In [21]:
"""
Função para descobrir se os dados do valor_total estão corretos. A lógica é:
    - valor_total consiste no valor_unitario multiplicado pela quantidade
    - se o resultado for zero, significa que não há inconsistências
    - caso contrário, há valores incorretos.
"""

# o abs transforma em valor absoluto, retirando o negativo.
(df['valor_unitario'] * df['quantidade'] - df['valor_total']).abs().max()

3.637978807091713e-12

# Análise

O resultado de $3.637978807091713e-12$ é extremamente pequeno, muito próximo de zero. Isso significa que não há inconsistência entre os valores unitários e total.

# Importando para BI

Para realizar a análise no **Power BI**, exportarei esse arquivo novamente como ```xlsx```.

In [33]:
import openpyxl

df.to_excel("./vendas_tratado.xlsx", index=False)